In [1]:
# Hidden cell for nbsphinx (check metadata)
import warnings
warnings.simplefilter('ignore')

# Quickstart

This notebook provides a quick introduction in using Carsus with the SQL interface.

## Creating a database

To start working with a database you need to initialize it. This requires an `url`, for example `sqlite:///path/to/example.db`. In this guide we'll use a in-memory `sqlite` database, so we leave the `url` empty:

In [2]:
from carsus import init_db
session = init_db()

Initializing the database at sqlite://
Ingesting basic atomic data


Now, we have an SQLAlchemy `Session` object named `session`. We can use `session` to make simple queries. As you can see from the output, some basic atomic data has been ingested into our database. Let's examine it:

In [3]:
from carsus.model import Atom

print('Number of Atoms in the database: {}'.format(session.query(Atom).count()))
si = session.query(Atom).get(14) 
print('Symbol: {}, atomic_number: {}, name: {}'.format(si.symbol, si.atomic_number, si.name))

Number of Atoms in the database: 118
Symbol: Si, atomic_number: 14, name: Silicon


So, our database already contains basic information about atoms. But this information has not been commited to the database yet. It is always **your responsibility** to commit the data!

In [4]:
session.commit()

## Atomic weights and ionization energies

To store more interesting data we are going to use **ingesters**. For each data source supported by Carsus there is a corresponding **ingester class**. 

For example, let's ingest atomic weights from the NIST Atomic Weights and Isotopic Compositions database.


In [5]:
from carsus.io.nist import NISTWeightsCompIngester

weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()

session.commit()

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets
Ingesting atomic weights from nist


In [6]:
print(si.weights)

[<Quantity: 28.085 u>]


Let's continue with ionization energies:

In [7]:
from carsus.io.nist import NISTIonizationEnergiesIngester

ioniz_energies_ingester = NISTIonizationEnergiesIngester(session, spectra='H-Mg')
ioniz_energies_ingester.ingest(ionization_energies=True, ground_levels=True)

session.commit()

Ingesting ionization energies from nist-asd
Ingesting ground levels from nist-asd


## Levels, lines and collisions

Currently, Carsus supports two sources of energy levels and transition lines: the Robert Kurucz’s line list (GFALL) and the Chianti Database.


### GFALL

In [8]:
from carsus.io.kurucz import GFALLIngester

gfall_url = 'http://kurucz.harvard.edu/linelists/gfall/gfall.dat'
gfall_ingester = GFALLIngester(session, fname=gfall_url, ions='H-Mg')
gfall_ingester.ingest(levels=True, lines=True)

session.commit()

[carsus.io.kurucz.gfall][WARNING]  A specific combination to identify unique levels from the gfall data has not been given. Defaulting to ["energy", "j"]. (gfall.py:77)
[carsus.io.kurucz.gfall][INFO   ]  Parsing GFALL http://kurucz.harvard.edu/linelists/gfall/gfall.dat (gfall.py:132)
Ingesting levels from ku_latest
Ingesting levels for He 0
Ingesting levels for He 1
Ingesting levels for Li 0
Ingesting levels for Li 1
Ingesting levels for Be 0
Ingesting levels for Be 1
Ingesting levels for Be 2
Ingesting levels for B 0
Ingesting levels for B 1
Ingesting levels for B 2
Ingesting levels for B 3
Ingesting levels for C 0
Ingesting levels for C 1
Ingesting levels for C 2
Ingesting levels for C 3
Ingesting levels for N 0
Ingesting levels for N 1
Ingesting levels for N 2
Ingesting levels for N 3
Ingesting levels for N 4
Ingesting levels for N 5
Ingesting levels for O 0
Ingesting levels for O 1
Ingesting levels for O 2
Ingesting levels for O 3
Ingesting levels for O 4
Ingesting levels for O 5
I

### CHIANTI

You can ingest `levels`, `lines` and `collisions` from the Chianti Database.

In [9]:
from carsus.io.chianti_ import ChiantiIngester

chianti_ingester = ChiantiIngester(session, ions='O 0-2')
chianti_ingester.ingest(levels=True, lines=True, collisions=True)

session.commit()

Ingesting levels from chianti_v9.0.1
Ingesting levels for O 0
Ingesting levels for O 1
Ingesting levels for O 2
Ingesting lines from chianti_v9.0.1
Ingesting lines for O 0
Ingesting lines for O 1
Ingesting lines for O 2
Ingesting collisions from chianti_v9.0.1
Ingesting collisions for O 0
Ingesting collisions for O 1
Ingesting collisions for O 2


## Zeta data

In [10]:
import os
import carsus
from carsus.io.zeta import KnoxLongZetaIngester

zeta_url = 'https://raw.githubusercontent.com/tardis-sn/carsus/master/carsus/data/knox_long_recombination_zeta.dat'
zeta_ingester = KnoxLongZetaIngester(session, zeta_url)
zeta_ingester.ingest()

session.commit()

## Save to HDF5

Once you have a database it is very easy to create atomic data files for TARDIS.

To do this, you need to use a special class called `AtomData`. This class takes `session` as its first argument; other important parameters are `selected_atoms` and `chianti_ions` (if any). 

Only data for `selected_atoms` will be stored in the output DataFrames and levels and lines for `chianti_ions` will be taken from the data source with the same name.

In [11]:
from carsus.io.output import AtomData
atom_data = AtomData(session, 
                     selected_atoms='H-Mg', 
                     chianti_short_name='chianti_v9.0.1', 
                     chianti_ions='O 0-2')

<div class="alert alert-warning">

**NOTE:** if `chianti_short_name` does NOT match with your current version of Chianti DB the above command will fail.

</div>


The stored `DataFrames` can be easily accessed as attributes of `atom_data`:

In [12]:
atom_data.atom_masses_prepared

,symbol,name,mass
atomic_number,,,
1,b'H',b'Hydrogen',1.007975
2,b'He',b'Helium',4.002602
3,b'Li',b'Lithium',6.967500
4,b'Be',b'Beryllium',9.012183
5,b'B',b'Boron',10.813500
6,b'C',b'Carbon',12.010600
7,b'N',b'Nitrogen',14.006855
8,b'O',b'Oxygen',15.999400
9,b'F',b'Fluorine',18.998403


In [13]:
atom_data.levels_prepared.head(20)

energy  g  metastable
atomic_number ion_number level_number                          
1             0          0              0.000000  2        True
              1          0              0.000000  1        True
2             0          0              0.000000  1        True
                         1             19.819626  3        True
                         2             20.615788  1        True
                         3             20.964099  5       False
                         4             20.964109  3       False
                         5             20.964117  5        True
                         6             20.964117  9        True
                         7             20.964231  1       False
                         8             21.218036  3       False
                         9             22.718479  3       False
                         10            22.920331  1       False
                         11            23.007085  5       False
                         12            23.007087  3       False
                         13            23.007090  5        True
                         14            23.007121  1       False
                         15            23.073663  7       False
                         16            23.073663  5       False
                         17            23.073664  7        True

In [14]:
atom_data.macro_atom_prepared.head(20)

,atomic_number,ion_number,source_level_number,destination_level_number,transition_type,transition_probability,transition_line_id
0,2,0,0,142,1,0.000000e+00,69
1,2,0,0,132,1,0.000000e+00,70
2,2,0,0,122,1,0.000000e+00,71
3,2,0,0,110,1,0.000000e+00,72
4,2,0,0,96,1,0.000000e+00,73
5,2,0,0,81,1,0.000000e+00,74
6,2,0,0,64,1,0.000000e+00,75
7,2,0,0,50,1,0.000000e+00,76
8,2,0,0,35,1,0.000000e+00,77
9,2,0,0,20,1,0.000000e+00,78


Finally, to create an HDF5 file you need to use the `to_hdf()` method.

In [15]:
atom_data.to_hdf('example_store.h5', 
                 store_atom_masses=True, 
                 store_ionization_energies=True, 
                 store_levels=True, 
                 store_lines=True,
                 store_collisions=True,
                 store_macro_atom=True,
                 store_zeta_data=True)

Signing AtomData: 
MD5: b0bfdc6f67be9dd7fec1aaffbd600c0d
UUID1: c9dbb3fcb5a311ea8f45c83dd46c2c53


You are done! Now you can use the created HDF5 atomic file to run TARDIS simulations.

In [16]:
# Hidden cell
!rm example_store.h5